#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [8]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

1. 뉴스기사 선언

In [9]:
article = """
가계대출 급증세가 이어지면서 정부가 서민들의 주택 구입과 전세 대출을 돕는 ‘디딤돌(구입)·버팀목(전세) 대출’ 금리를 최대 0.4%포인트 인상했습니다.

정책성 대출이 가계대출 증가에 ‘불쏘시개’ 역할을 했다는 지적에 따른 조치입니다. 그러나 집값 상승이 고가 주택 위주로 이뤄진 상황에서, 서민들만 옥죄는 게 아니냐는 지적도 나옵니다. 

국토교통부와 금융권에 따르면 지난 16일부터 주택도시기금의 대출금리와 시중금리간 적정한 차이를 유지할 필요가 있다는 점을 감안해 디딤돌·버팀목 대출금리를 0.2∼0.4% 포인트 인상했습니다. 디딤돌대출(주택 매매) 금리는 기존 연 2.15~3.55%에서 2.35~3.95%로, 버팀목대출(전세) 금리는 기존 1.5~2.9%에서 1.7~3.3%로 올랐습니다. 이들 대출은 주택도시기금 재원으로 공급되는 정책금융상품인데, 서민 대출자들이 내야 할 이자를 늘려 규제를 가한 것입니다.

정부가 이처럼 정책대출의 금리를 높인 것은 시중은행 대비 금리가 낮은 디딤돌 등 정책 상품에 대출 수요가 몰리면서, 가계 대출 및 집값 상승세를 자극하고 있다고 보고 있기 때문입니다. 실제 디딤돌 대출의 경우 올 상반기 공급액(집행 실적 기준)이 약 15조원으로 지난해 상반기(8조2000억원)대비 1.8배 늘었습니다. 

하지만 디딤돌 대출 수요자들 사이에서는 집값 상승의 원인은 6억원 초과의 중고가 아파트에 있는데 왜 서민 주거 안정을 위한 디딤돌 대출부터 옥죄냐는 불만이 나오고 있습니다. 

실제 직방 자료를 보면 올해 상반기(1~6월) 서울 아파트 매매거래 2만3328건 가운데 53.1%(1만2396건)가 9억원 초과 거래로, 이는 통계가 집계된 2006년 이래 가장 높은 수준입니다. 지난해 하반기(7964건)와 비교하면 반년 사이 55.7%나 급증했습니다. 같은 기간 6억원 초과~9억원 이하와 3억원 초과~6억원 이하 거래는 각각 37%, 21.2% 증가에 불과했습니다. 

경제만랩이 국토부의 실거래가를 분석한 자료에서도 서울 아파트 매매 가운데 디딤돌·버팀목 대출을 받을 수 있는 6억원 미만 거래의 비율은 올 1분기 역대 최저 수준을 기록한 반면, 15억원 이상 아파트 거래량은 역대 최고 비율을 보였습니다. 
"""

2. 텍스트 전처리

In [10]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 가계 대출 급증 세 정부 서민 주택 구입 전세 대출 디딤돌 구입 버팀목 전세 대출 금리 최대 포인트 인상 정책 대출 가계 대출 증가 불쏘시개 역할 지적 조치 집값 상승 고가 주택 위주 상황 서민 옥죄 게 지적도 국토교통부 금융 지난 주택 기금 대출 금리 시중금리 차이 유지 필요 점 감안 디딤돌 버팀목 대출 금리 포인트 인상 디딤돌대출 주택 매매 금리 기존 연 로 버팀목 대출 전세 금리 기존 로 이 대출 주택 도시 기금 재원 공급 정책금융 상품 서민 출자 내야 이자 규제 것 정부 이처럼 정책 대출 금리 것 시중 은행 대비 금리 디딤돌 등 정책 상품 대출 수요 가계 대출 및 집값 상승세 자극 보고 때문 실제 디딤돌 대출 경우 상반기 공급 액 집행 실적 기준 이 약 지난해 상반기 대비 배 디딤돌 대출 자 사이 집값 상승 원인 초과 중고 아파트 왜 서민 주거 안정 위 디딤돌 대출 옥죄 불만 실제 직방 자료 올해 상반기 서울 아파트 매매 거래 건 가운데 건 초과 거래 통계 집계 가장 수준 지난해 하반기 건 비교 반년 사이 나 급증 기간 초과 이하 초과 이하 거래 각각 증가 경제 랩 국토부 실 거래 분석 자료 서울 아파트 매매 가운데 디딤돌 버팀목 대출 수 미만 거래 비율 역대 최저 수준 기록 반면 이상 아파트 거래량 역대 최고 비율


3. N-Gram 생성 및 SBERT

In [11]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 263
N-Gram 다섯개 출력 : ['가계' '가계 대출' '가운데' '가운데 디딤돌' '가운데 초과']


c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SBERT 분석 후 뉴스 기사 : [[ 4.80405331e-01  4.50041145e-04  9.10722196e-01 -7.07808286e-02
  -3.37114125e-01  8.69305208e-02 -3.34426276e-02 -2.54705757e-01
  -3.05445433e-01  1.63440019e-01 -2.72608489e-01  8.04110289e-01
  -1.10112824e-01 -3.54715884e-02 -6.27855241e-01 -8.72527510e-02
   2.22506195e-01 -9.00038704e-02  8.03746134e-02 -1.52940854e-01
   1.99068695e-01 -6.54241562e-01  9.74454999e-01  6.08868003e-01
  -3.57717536e-02  3.91618945e-02 -1.53548405e-01  6.87346339e-01
   2.63304822e-03 -3.90738487e-01  9.82297808e-02  2.02268198e-01
  -2.65161395e-01 -1.81753844e-01  5.85375667e-01  6.50824070e-01
  -2.15094894e-01  7.26619735e-02 -2.68829942e-01 -5.57100415e-01
   3.08103979e-01 -1.93529874e-01 -3.01265657e-01 -4.45628092e-02
  -6.32875919e-01 -1.18232764e-01 -7.29566038e-01  6.64955318e-01
  -1.97555080e-01 -3.65555286e-01  5.60338378e-01  2.89941132e-01
  -1.87207788e-01 -9.53284264e-01 -3.79061610e-01  5.69335461e-01
  -5.44902980e-01 -6.24916434e-01  1.58682555e-01 -4.3806

4. Max Sum Similarity

In [12]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['고가 주택', '기존 대출', '대출 수요', '증가 경제', '집값 상승세']


5. 테마 가져오기

In [13]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [14]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
print(f'뉴스 기사 : \n{article}')
print(f'뉴스 키워드 5개 : {news_keywords}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {loaded_dict[cosine_recommend[-i]]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1

뉴스 기사 : 
가계대출 급증세가 이어지면서 정부가 서민들의 주택 구입과 전세 대출을 돕는 ‘디딤돌(구입)·버팀목(전세) 대출’ 금리를 최대 0.
4%포인트 인상했습니다.
정책성 대출이 가계대출 증가에 ‘불쏘시개’ 역할을 했다는 지적에 따른 조치입니다.
 그러나 집값 상승이 고가 주택 위주로 이뤄진 상황에서, 서민들만 옥죄는 게 아니냐는 지적도 나옵니다.
 국토교통부와 금융권에 따르면 지난 16일부터 주택도시기금의 대출금리와 시중금리간 적정한 차이를 유지할 필요가 있다는 점을 감안해 디딤돌·버팀목 대출금리를 0.
2∼0.
4% 포인트 인상했습니다.
 디딤돌대출(주택 매매) 금리는 기존 연 2.
15~3.
55%에서 2.
35~3.
95%로, 버팀목대출(전세) 금리는 기존 1.
5~2.
9%에서 1.
7~3.
3%로 올랐습니다.
 이들 대출은 주택도시기금 재원으로 공급되는 정책금융상품인데, 서민 대출자들이 내야 할 이자를 늘려 규제를 가한 것입니다.
정부가 이처럼 정책대출의 금리를 높인 것은 시중은행 대비 금리가 낮은 디딤돌 등 정책 상품에 대출 수요가 몰리면서, 가계 대출 및 집값 상승세를 자극하고 있다고 보고 있기 때문입니다.
 실제 디딤돌 대출의 경우 올 상반기 공급액(집행 실적 기준)이 약 15조원으로 지난해 상반기(8조2000억원)대비 1.
8배 늘었습니다.
 하지만 디딤돌 대출 수요자들 사이에서는 집값 상승의 원인은 6억원 초과의 중고가 아파트에 있는데 왜 서민 주거 안정을 위한 디딤돌 대출부터 옥죄냐는 불만이 나오고 있습니다.
 실제 직방 자료를 보면 올해 상반기(1~6월) 서울 아파트 매매거래 2만3328건 가운데 53.
1%(1만2396건)가 9억원 초과 거래로, 이는 통계가 집계된 2006년 이래 가장 높은 수준입니다.
 지난해 하반기(7964건)와 비교하면 반년 사이 55.
7%나 급증했습니다.
 같은 기간 6억원 초과~9억원 이하와 3억원 초과~6억원 이하 거래는 각각 37%, 21.
2% 증가에 불과했습니다.
 경제만랩이 국토부의 실거래가